In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import pandas as pd
import numpy as np
from gtda.time_series import SlidingWindow
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.keras.backend import set_session
config = tf.compat.v1.ConfigProto() 
config.gpu_options.allow_growth = True  
config.log_device_placement = True  
sess2 = tf.compat.v1.Session(config=config)
set_session(sess2) 
import tensorflow_datasets as tfds
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MSE
from tqdm import tqdm

## Import Dataset

In [ ]:
dataset_name = "cifar10"
(ds_train, ds_test), ds_info = tfds.load(
    dataset_name, split=["train", "test"], with_info=True, as_supervised=True
)
NUM_CLASSES = ds_info.features["label"].num_classes

IMG_SIZE = 32
batch_size = 1
size = (IMG_SIZE, IMG_SIZE)
ds_train = ds_train.map(lambda image, label: (tf.image.resize(image, size), label))
ds_test = ds_test.map(lambda image, label: (tf.image.resize(image, size), label))

def input_preprocess(image, label):
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label


ds_train = ds_train.map(
    input_preprocess, num_parallel_calls=tf.data.AUTOTUNE
)
ds_train = ds_train.batch(batch_size=batch_size, drop_remainder=True)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(input_preprocess)
ds_test = ds_test.batch(batch_size=batch_size, drop_remainder=True)

## Load Model

In [ ]:
model = load_model('/home/nesl/209as_sec/cifar10/TinyML Models/ResNet8/pretrainedResnet.h5')
model.summary()

## Attack

In [ ]:
def pgd_attack(model,iterations, image, label, alpha, eps):
    gen_img = tf.identity(image)
    gen_img = gen_img + tf.random.uniform(gen_img.get_shape().as_list(), minval=-eps, 
                                          maxval=eps, dtype=tf.dtypes.float32)
    x_temp = image
    for iter in range(iterations):
        imgv = tf.Variable(gen_img)
        with tf.GradientTape() as tape:
            tape.watch(imgv)
            predictions = model(imgv)
            loss = tf.keras.losses.CategoricalCrossentropy()(label, predictions)
            grads = tape.gradient(loss,imgv)
        signed_grads = tf.sign(grads)
        gen_img = gen_img + (alpha*signed_grads)
        gen_img = tf.clip_by_value(gen_img, image-eps, image+eps)
    return gen_img

In [ ]:
eps = [0.1,0.3,0.5,0.7,0.9,1.0,2.0]
iterations = 5
alpha = [0.1,0.3,0.5,0.7,0.9,1.0]
take_size = 1000
accu_num = []
eps_list = []
alpha_list = []

for al in alpha:
    for item in eps:
        countadv = 0
        for image, label in tqdm(ds_test.take(take_size)):
            imageLabel = np.array(label).argmax()
            imagePred = model.predict(image)
            imagePred = imagePred.argmax()
            adversary = pgd_attack(model,iterations,image, label, alpha=al, eps=item)
            pred = model.predict(adversary)
            adversaryPred = pred[0].argmax()
            if imagePred == adversaryPred:
                countadv += 1
            
        print("Adversarial accuracy : ", countadv / take_size)
        accu_num.append(countadv / take_size)
        eps_list.append(item)
        alpha_list.append(al)

In [ ]:
accu_num

In [ ]:
eps_list

In [ ]:
alpha_list